In [1]:
# All library calls go here
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc #mamba install -c conda-forge netCDF4 # conda install -c conda-forge netcdf4

import planetary_computer
import xarray as xr
import fsspec
import pystac_client
import datetime as dtt

import requests

from matplotlib import pyplot

import dask_gateway

import pickle

import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient,ContentSettings
import io

In [2]:
# Read in location data
locationsfile="AllCoffeeLocations150623.csv"
locDF = pd.read_csv(locationsfile)
newLons=locDF["longitude"].copy()
newLons[newLons<0.0]=newLons[newLons<0.0]+360.0
locDF["newLongitude"]=newLons
locDF

,SiteID,latitude,longitude,Source,Species,newLongitude
0,0,-19.125000,-39.791667,Monfredaetal2008,Unknown,320.208333
1,1,11.875000,76.041667,Monfredaetal2008,Unknown,76.041667
2,2,11.791667,76.125000,Monfredaetal2008,Unknown,76.125000
3,3,11.625000,76.291667,Monfredaetal2008,Unknown,76.291667
4,4,11.708333,76.375000,Monfredaetal2008,Unknown,76.375000
...,...,...,...,...,...,...
123,123,15.676065,-91.766889,Mercon,Arabica,268.233111
124,124,15.619671,-91.693777,Mercon,Arabica,268.306223
125,125,15.611956,-91.696036,Mercon,Arabica,268.303964
126,126,15.676866,-91.751296,Mercon,Arabica,268.248704


In [3]:
#Towards reading in climate data, set up STAC
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("nasa-nex-gddp-cmip6")

In [4]:
m='ACCESS-CM2'
var='tas'
scen='ssp585'
search = catalog.search(
    collections=["nasa-nex-gddp-cmip6"],
    datetime="2015/2040",
    query={"cmip6:model": {"eq": m},"cmip6:scenario": {"eq": scen}},
)

In [5]:
items = search.item_collection()
tsa = xr.open_mfdataset(
    [fsspec.open(item.assets[var].href).open() for item in items],
)

In [6]:
cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
cluster.adapt(minimum=2, maximum=100)
print(cluster.dashboard_link)
ts2=tsa[var].sel(lat=xr.DataArray(locDF['latitude']), lon=xr.DataArray(locDF['newLongitude']),method="nearest").compute()
cluster.close()

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.4fffd1fe0250468792ea2fbb035cfe9e/status
